In [167]:
import face_recognition as fr
import numpy as np
import os
import tqdm
from scipy.spatial.distance import pdist, squareform
import shutil

# Embeddings
Get embeddings for all faces in the dataset.

In [12]:
dir_face_origin = os.path.join("origin", "face")
files_face = [file for file in os.listdir(dir_face_origin) if file.endswith(".tif")]
ebds_list = []
for file_face in tqdm.tqdm(files_face):
    ebds = fr.face_encodings(fr.load_image_file(os.path.join(dir_face_origin, file_face)))
    ebds_list.append(ebds)
ebds_mat = np.concatenate(ebds_list, axis=0)
simils = 1 - squareform(pdist(ebds_mat, 'cosine'))

100%|██████████| 796/796 [03:36<00:00,  3.67it/s]


In [173]:
# np.save("ebds_mat.npy", ebds_mat)
ebds_mat = np.load("ebds_mat.npy")

array([[-0.07574854,  0.0631862 , -0.00545767, ..., -0.0455986 ,
         0.08221815, -0.05544692],
       [-0.07301039,  0.12866941,  0.05676532, ..., -0.04168748,
         0.025063  , -0.06077523],
       [-0.09061348,  0.11383098, -0.00125085, ..., -0.02179576,
         0.06842856,  0.03265785],
       ...,
       [-0.12096058,  0.07091207,  0.04049471, ..., -0.00586712,
         0.06068014,  0.019821  ],
       [-0.18452413,  0.0516095 ,  0.0278815 , ..., -0.06559899,
         0.0904255 , -0.00537251],
       [-0.10438296,  0.05099077,  0.0278545 , ..., -0.0140014 ,
         0.07005149, -0.01001175]])

# Select morphing faces

In [170]:
from numpy.random import default_rng
rng = default_rng(seed=4)
# choose randomly
indices_from = rng.choice(len(ebds_list), size=60, replace=False)
simils_from = simils[indices_from, :]
# ensure morph faces are not too similar to chosen faces
indices_to_pool = np.sort(np.where(np.all(simils_from < 0.95, axis=0))[0])
indices_to = np.ndarray(shape=(60,), dtype=int)
for i in range(len(indices_from)):
    index_to = np.argmin(simils_from[i, indices_to_pool])
    indices_to[i] = indices_to_pool[index_to]
    indices_to_pool = np.delete(indices_to_pool, index_to)

files_morph_from = [files_face[index] for index in indices_from]
files_morph_to = [files_face[index] for index in indices_to]
simil_sel = simils[np.ix_(indices_from, indices_to)]
file_order = np.argsort(np.diag(simil_sel))
# side effect: copy files
for i in range(len(files_morph_from)):
    shutil.copy(os.path.join(dir_face_origin, files_morph_from[i]), os.path.join("working", "face", f'{file_order[i]+1:03d}_from.tif'))
    shutil.copy(os.path.join(dir_face_origin, files_morph_to[i]), os.path.join("working", "face", f'{file_order[i]+1:03d}_to.tif'))

# Select Novel Faces

In [171]:
files_face_remain = [file for file in files_face if file not in files_morph_from and file not in files_morph_to]
indices_novel = rng.choice(len(files_face_remain), size=45, replace=False)
files_novel = [files_face_remain[index] for index in indices_novel]
for i in range(len(files_novel)):
    shutil.copy(os.path.join(dir_face_origin, files_novel[i]), os.path.join("working", "face", f'{i+1:03d}_novel.tif'))